In [1]:
from cascade.model import CascadeNet
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
from utils import *
from mri_utils import *
import os
import torchvision
from skimage.measure import compare_ssim as ssim

In [2]:
G = CascadeNet().cuda() # input channels:  Z(X) x Y -> output channel X

In [3]:
mri_dataset = MRIDataSet('../data/CORRECTED_COMBINED_bilkent_60_training.mat')
dataloader = DataLoader(mri_dataset, batch_size=1, shuffle=True)
# x, y, mask = next(iter(dataloader_test))
# x = torch.load('../data/x.pt')

In [4]:
def psnr(img, r_img):
    return 10*torch.log10(torch.max(r_img)**2/torch.mean((img-r_img)**2))

In [5]:
idx =1999
G.load_state_dict(torch.load('./parameters/checkpoint%d/G.pt'%(idx)))

In [6]:
def uncertainty_1d(y, mask):
    '''
    >>> y: subsampled image(spatial domain) (1, 2, 256, 256)
    >>> mask: 2d mask(256, 256)
    return
    >>> uncertainty along the vertical axis (256, )
    '''
    fs = []
    for i in range(2):
        zv = torch.rand((1, 2, 256, 256)).cuda()
        observedv = y.cuda()
        inpv = torch.cat((observedv, zv), 1)
        fs.append(torch.fft(to5d(G(inpv, torch.unsqueeze(mask, 0).cuda()).cpu().detach()), signal_ndim = 2,normalized=True))

    fs = torch.cat(fs, 0)
    f_var = torch.var(fs, 0)
    f_var = f_var[0,:,:,0] +  f_var[0,:,:,1]
#     f_std = torch.sqrt(f_var)
    return torch.sum(f_var, 1)

In [7]:
def apply_mask(x, mask):
    '''
    >>> x : full scan MRI image(spatial domain) (1, 2, 256, 256)
    >>> mask: 2d mask (H, W)
    
    return
    >>> subsampled image(spatial domain) (1,2, 256, 256)
    '''
    return backward_op(forward_op(to5d(x),mask))

In [8]:
idx = [] # indices where a line will be put
mask = torch.zeros((256, 256)) # initial mask(all zeros)
masks = []
uncers = []
rate = np.round([0.1*256, 0.15*256, 0.2*256, 0.25*256, 0.3*256, 0.35*256, 0.4*256, 0.45*256, 0.5*256])
for i in range(128):
    uncer = 0
    for j, (x, _, _) in enumerate(dataloader):
        y = apply_mask(x, mask)
        uncer += uncertainty_1d(y, mask)
        print('line %d/%d data %d/300'%(i, 256, j))
    uncers.append(uncer)
    idx.append(torch.argmax(uncer).item())
#     idxs.append(idx)
    mask[idx, :] = 1
    print('average uncertainty: %f'%(torch.sum(uncer)/300))

line 0/256 data 0/300
line 0/256 data 1/300
line 0/256 data 2/300
line 0/256 data 3/300
line 0/256 data 4/300
line 0/256 data 5/300
line 0/256 data 6/300
line 0/256 data 7/300
line 0/256 data 8/300
line 0/256 data 9/300
line 0/256 data 10/300
line 0/256 data 11/300
line 0/256 data 12/300
line 0/256 data 13/300
line 0/256 data 14/300
line 0/256 data 15/300
line 0/256 data 16/300
line 0/256 data 17/300
line 0/256 data 18/300
line 0/256 data 19/300
line 0/256 data 20/300
line 0/256 data 21/300
line 0/256 data 22/300
line 0/256 data 23/300
line 0/256 data 24/300
line 0/256 data 25/300
line 0/256 data 26/300
line 0/256 data 27/300
line 0/256 data 28/300
line 0/256 data 29/300
line 0/256 data 30/300
line 0/256 data 31/300
line 0/256 data 32/300
line 0/256 data 33/300
line 0/256 data 34/300
line 0/256 data 35/300
line 0/256 data 36/300
line 0/256 data 37/300
line 0/256 data 38/300
line 0/256 data 39/300
line 0/256 data 40/300
line 0/256 data 41/300
line 0/256 data 42/300
line 0/256 data 43/30

line 1/256 data 48/300
line 1/256 data 49/300
line 1/256 data 50/300
line 1/256 data 51/300
line 1/256 data 52/300
line 1/256 data 53/300
line 1/256 data 54/300
line 1/256 data 55/300
line 1/256 data 56/300
line 1/256 data 57/300
line 1/256 data 58/300
line 1/256 data 59/300
line 1/256 data 60/300
line 1/256 data 61/300
line 1/256 data 62/300
line 1/256 data 63/300
line 1/256 data 64/300
line 1/256 data 65/300
line 1/256 data 66/300
line 1/256 data 67/300
line 1/256 data 68/300
line 1/256 data 69/300
line 1/256 data 70/300
line 1/256 data 71/300
line 1/256 data 72/300
line 1/256 data 73/300
line 1/256 data 74/300
line 1/256 data 75/300
line 1/256 data 76/300
line 1/256 data 77/300
line 1/256 data 78/300
line 1/256 data 79/300
line 1/256 data 80/300
line 1/256 data 81/300
line 1/256 data 82/300
line 1/256 data 83/300
line 1/256 data 84/300
line 1/256 data 85/300
line 1/256 data 86/300
line 1/256 data 87/300
line 1/256 data 88/300
line 1/256 data 89/300
line 1/256 data 90/300
line 1/256 

line 2/256 data 95/300
line 2/256 data 96/300
line 2/256 data 97/300
line 2/256 data 98/300
line 2/256 data 99/300
line 2/256 data 100/300
line 2/256 data 101/300
line 2/256 data 102/300
line 2/256 data 103/300
line 2/256 data 104/300
line 2/256 data 105/300
line 2/256 data 106/300
line 2/256 data 107/300
line 2/256 data 108/300
line 2/256 data 109/300
line 2/256 data 110/300
line 2/256 data 111/300
line 2/256 data 112/300
line 2/256 data 113/300
line 2/256 data 114/300
line 2/256 data 115/300
line 2/256 data 116/300
line 2/256 data 117/300
line 2/256 data 118/300
line 2/256 data 119/300
line 2/256 data 120/300
line 2/256 data 121/300
line 2/256 data 122/300
line 2/256 data 123/300
line 2/256 data 124/300
line 2/256 data 125/300
line 2/256 data 126/300
line 2/256 data 127/300
line 2/256 data 128/300
line 2/256 data 129/300
line 2/256 data 130/300
line 2/256 data 131/300
line 2/256 data 132/300
line 2/256 data 133/300
line 2/256 data 134/300
line 2/256 data 135/300
line 2/256 data 136/3

line 3/256 data 141/300
line 3/256 data 142/300
line 3/256 data 143/300
line 3/256 data 144/300
line 3/256 data 145/300
line 3/256 data 146/300
line 3/256 data 147/300
line 3/256 data 148/300
line 3/256 data 149/300
line 3/256 data 150/300
line 3/256 data 151/300
line 3/256 data 152/300
line 3/256 data 153/300
line 3/256 data 154/300
line 3/256 data 155/300
line 3/256 data 156/300
line 3/256 data 157/300
line 3/256 data 158/300
line 3/256 data 159/300
line 3/256 data 160/300
line 3/256 data 161/300
line 3/256 data 162/300
line 3/256 data 163/300
line 3/256 data 164/300
line 3/256 data 165/300
line 3/256 data 166/300
line 3/256 data 167/300
line 3/256 data 168/300
line 3/256 data 169/300
line 3/256 data 170/300
line 3/256 data 171/300
line 3/256 data 172/300
line 3/256 data 173/300
line 3/256 data 174/300
line 3/256 data 175/300
line 3/256 data 176/300
line 3/256 data 177/300
line 3/256 data 178/300
line 3/256 data 179/300
line 3/256 data 180/300
line 3/256 data 181/300
line 3/256 data 

line 4/256 data 187/300
line 4/256 data 188/300
line 4/256 data 189/300
line 4/256 data 190/300
line 4/256 data 191/300
line 4/256 data 192/300
line 4/256 data 193/300
line 4/256 data 194/300
line 4/256 data 195/300
line 4/256 data 196/300
line 4/256 data 197/300
line 4/256 data 198/300
line 4/256 data 199/300
line 4/256 data 200/300
line 4/256 data 201/300
line 4/256 data 202/300
line 4/256 data 203/300
line 4/256 data 204/300
line 4/256 data 205/300
line 4/256 data 206/300
line 4/256 data 207/300
line 4/256 data 208/300
line 4/256 data 209/300
line 4/256 data 210/300
line 4/256 data 211/300
line 4/256 data 212/300
line 4/256 data 213/300
line 4/256 data 214/300
line 4/256 data 215/300
line 4/256 data 216/300
line 4/256 data 217/300
line 4/256 data 218/300
line 4/256 data 219/300
line 4/256 data 220/300
line 4/256 data 221/300
line 4/256 data 222/300
line 4/256 data 223/300
line 4/256 data 224/300
line 4/256 data 225/300
line 4/256 data 226/300
line 4/256 data 227/300
line 4/256 data 

...

line 123/256 data 145/300
line 123/256 data 146/300
line 123/256 data 147/300
line 123/256 data 148/300
line 123/256 data 149/300
line 123/256 data 150/300
line 123/256 data 151/300
line 123/256 data 152/300
line 123/256 data 153/300
line 123/256 data 154/300
line 123/256 data 155/300
line 123/256 data 156/300
line 123/256 data 157/300
line 123/256 data 158/300
line 123/256 data 159/300
line 123/256 data 160/300
line 123/256 data 161/300
line 123/256 data 162/300
line 123/256 data 163/300
line 123/256 data 164/300
line 123/256 data 165/300
line 123/256 data 166/300
line 123/256 data 167/300
line 123/256 data 168/300
line 123/256 data 169/300
line 123/256 data 170/300
line 123/256 data 171/300
line 123/256 data 172/300
line 123/256 data 173/300
line 123/256 data 174/300
line 123/256 data 175/300
line 123/256 data 176/300
line 123/256 data 177/300
line 123/256 data 178/300
line 123/256 data 179/300
line 123/256 data 180/300
line 123/256 data 181/300
line 123/256 data 182/300
line 123/256

line 124/256 data 164/300
line 124/256 data 165/300
line 124/256 data 166/300
line 124/256 data 167/300
line 124/256 data 168/300
line 124/256 data 169/300
line 124/256 data 170/300
line 124/256 data 171/300
line 124/256 data 172/300
line 124/256 data 173/300
line 124/256 data 174/300
line 124/256 data 175/300
line 124/256 data 176/300
line 124/256 data 177/300
line 124/256 data 178/300
line 124/256 data 179/300
line 124/256 data 180/300
line 124/256 data 181/300
line 124/256 data 182/300
line 124/256 data 183/300
line 124/256 data 184/300
line 124/256 data 185/300
line 124/256 data 186/300
line 124/256 data 187/300
line 124/256 data 188/300
line 124/256 data 189/300
line 124/256 data 190/300
line 124/256 data 191/300
line 124/256 data 192/300
line 124/256 data 193/300
line 124/256 data 194/300
line 124/256 data 195/300
line 124/256 data 196/300
line 124/256 data 197/300
line 124/256 data 198/300
line 124/256 data 199/300
line 124/256 data 200/300
line 124/256 data 201/300
line 124/256

line 125/256 data 183/300
line 125/256 data 184/300
line 125/256 data 185/300
line 125/256 data 186/300
line 125/256 data 187/300
line 125/256 data 188/300
line 125/256 data 189/300
line 125/256 data 190/300
line 125/256 data 191/300
line 125/256 data 192/300
line 125/256 data 193/300
line 125/256 data 194/300
line 125/256 data 195/300
line 125/256 data 196/300
line 125/256 data 197/300
line 125/256 data 198/300
line 125/256 data 199/300
line 125/256 data 200/300
line 125/256 data 201/300
line 125/256 data 202/300
line 125/256 data 203/300
line 125/256 data 204/300
line 125/256 data 205/300
line 125/256 data 206/300
line 125/256 data 207/300
line 125/256 data 208/300
line 125/256 data 209/300
line 125/256 data 210/300
line 125/256 data 211/300
line 125/256 data 212/300
line 125/256 data 213/300
line 125/256 data 214/300
line 125/256 data 215/300
line 125/256 data 216/300
line 125/256 data 217/300
line 125/256 data 218/300
line 125/256 data 219/300
line 125/256 data 220/300
line 125/256

line 126/256 data 202/300
line 126/256 data 203/300
line 126/256 data 204/300
line 126/256 data 205/300
line 126/256 data 206/300
line 126/256 data 207/300
line 126/256 data 208/300
line 126/256 data 209/300
line 126/256 data 210/300
line 126/256 data 211/300
line 126/256 data 212/300
line 126/256 data 213/300
line 126/256 data 214/300
line 126/256 data 215/300
line 126/256 data 216/300
line 126/256 data 217/300
line 126/256 data 218/300
line 126/256 data 219/300
line 126/256 data 220/300
line 126/256 data 221/300
line 126/256 data 222/300
line 126/256 data 223/300
line 126/256 data 224/300
line 126/256 data 225/300
line 126/256 data 226/300
line 126/256 data 227/300
line 126/256 data 228/300
line 126/256 data 229/300
line 126/256 data 230/300
line 126/256 data 231/300
line 126/256 data 232/300
line 126/256 data 233/300
line 126/256 data 234/300
line 126/256 data 235/300
line 126/256 data 236/300
line 126/256 data 237/300
line 126/256 data 238/300
line 126/256 data 239/300
line 126/256

line 127/256 data 221/300
line 127/256 data 222/300
line 127/256 data 223/300
line 127/256 data 224/300
line 127/256 data 225/300
line 127/256 data 226/300
line 127/256 data 227/300
line 127/256 data 228/300
line 127/256 data 229/300
line 127/256 data 230/300
line 127/256 data 231/300
line 127/256 data 232/300
line 127/256 data 233/300
line 127/256 data 234/300
line 127/256 data 235/300
line 127/256 data 236/300
line 127/256 data 237/300
line 127/256 data 238/300
line 127/256 data 239/300
line 127/256 data 240/300
line 127/256 data 241/300
line 127/256 data 242/300
line 127/256 data 243/300
line 127/256 data 244/300
line 127/256 data 245/300
line 127/256 data 246/300
line 127/256 data 247/300
line 127/256 data 248/300
line 127/256 data 249/300
line 127/256 data 250/300
line 127/256 data 251/300
line 127/256 data 252/300
line 127/256 data 253/300
line 127/256 data 254/300
line 127/256 data 255/300
line 127/256 data 256/300
line 127/256 data 257/300
line 127/256 data 258/300
line 127/256

In [16]:
######## stochastic version ##########
idx = [] # indices where a line will be put
# masks = []
uncers = []
rate = np.round([0.1*256, 0.15*256, 0.2*256, 0.25*256, 0.3*256, 0.35*256, 0.4*256, 0.45*256, 0.5*256])
batchsize = 5
lines = 128
for i in range(lines):
    batch = mri_dataset.random_get(batchsize)
    uncer = 0
    for j, x in enumerate(batch):
        x = x.unsqueeze(0)
        y = apply_mask(x, mask)
        uncer += uncertainty_1d(y, mask)
        print('line %d/%d data %d/%d'%(i, lines, j, batchsize))
    uncers.append(uncer)
    idx.append(torch.argmax(uncer).item())
    print('average uncertainty: %f'%(torch.sum(uncer)/batchsize))
#     if (i+1) in rate:
#         masks.append(mask)
#         print('mask %d saved'%(i+1))

line 0/128 data 0/5
line 0/128 data 1/5
line 0/128 data 2/5
line 0/128 data 3/5
line 0/128 data 4/5
average uncertainty: 9.776624
line 1/128 data 0/5
line 1/128 data 1/5
line 1/128 data 2/5
line 1/128 data 3/5
line 1/128 data 4/5
average uncertainty: 12.002092
line 2/128 data 0/5
line 2/128 data 1/5
line 2/128 data 2/5
line 2/128 data 3/5
line 2/128 data 4/5
average uncertainty: 82.256653
line 3/128 data 0/5
line 3/128 data 1/5
line 3/128 data 2/5
line 3/128 data 3/5
line 3/128 data 4/5
average uncertainty: 119.149071
line 4/128 data 0/5
line 4/128 data 1/5
line 4/128 data 2/5
line 4/128 data 3/5
line 4/128 data 4/5
average uncertainty: 43.541698
line 5/128 data 0/5
line 5/128 data 1/5
line 5/128 data 2/5
line 5/128 data 3/5
line 5/128 data 4/5
average uncertainty: 47.512707
line 6/128 data 0/5
line 6/128 data 1/5
line 6/128 data 2/5
line 6/128 data 3/5
line 6/128 data 4/5
average uncertainty: 59.114361
line 7/128 data 0/5
line 7/128 data 1/5
line 7/128 data 2/5
line 7/128 data 3/5
lin

line 60/128 data 4/5
average uncertainty: 11.649932
line 61/128 data 0/5
line 61/128 data 1/5
line 61/128 data 2/5
line 61/128 data 3/5
line 61/128 data 4/5
average uncertainty: 10.063534
line 62/128 data 0/5
line 62/128 data 1/5
line 62/128 data 2/5
line 62/128 data 3/5
line 62/128 data 4/5
average uncertainty: 9.616931
line 63/128 data 0/5
line 63/128 data 1/5
line 63/128 data 2/5
line 63/128 data 3/5
line 63/128 data 4/5
average uncertainty: 10.858447
line 64/128 data 0/5
line 64/128 data 1/5
line 64/128 data 2/5
line 64/128 data 3/5
line 64/128 data 4/5
average uncertainty: 12.351134
line 65/128 data 0/5
line 65/128 data 1/5
line 65/128 data 2/5
line 65/128 data 3/5
line 65/128 data 4/5
average uncertainty: 9.061827
line 66/128 data 0/5
line 66/128 data 1/5
line 66/128 data 2/5
line 66/128 data 3/5
line 66/128 data 4/5
average uncertainty: 11.150743
line 67/128 data 0/5
line 67/128 data 1/5
line 67/128 data 2/5
line 67/128 data 3/5
line 67/128 data 4/5
average uncertainty: 8.821750

line 120/128 data 4/5
average uncertainty: 4.071033
line 121/128 data 0/5
line 121/128 data 1/5
line 121/128 data 2/5
line 121/128 data 3/5
line 121/128 data 4/5
average uncertainty: 3.631343
line 122/128 data 0/5
line 122/128 data 1/5
line 122/128 data 2/5
line 122/128 data 3/5
line 122/128 data 4/5
average uncertainty: 3.324696
line 123/128 data 0/5
line 123/128 data 1/5
line 123/128 data 2/5
line 123/128 data 3/5
line 123/128 data 4/5
average uncertainty: 3.445994
line 124/128 data 0/5
line 124/128 data 1/5
line 124/128 data 2/5
line 124/128 data 3/5
line 124/128 data 4/5
average uncertainty: 3.276463
line 125/128 data 0/5
line 125/128 data 1/5
line 125/128 data 2/5
line 125/128 data 3/5
line 125/128 data 4/5
average uncertainty: 2.735116
line 126/128 data 0/5
line 126/128 data 1/5
line 126/128 data 2/5
line 126/128 data 3/5
line 126/128 data 4/5
average uncertainty: 3.621074
line 127/128 data 0/5
line 127/128 data 1/5
line 127/128 data 2/5
line 127/128 data 3/5
line 127/128 data 4/